In [1]:
import os
%pwd

'/home/towet/Desktop/Visions/OpenProjects/Malaria-Cell-Classification/research'

In [2]:
os.chdir('../')

In [26]:
%pwd

'/home/towet/Desktop/Visions/OpenProjects/Malaria-Cell-Classification'

In [27]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path 
    model_path: Path 
    epochs: int
    batch_size: int
    lr: float 
    imgsz: int 
    include_top: bool 
    weights: str
    dataset: Path

In [5]:
from MalariaClassifier.constants import *
from MalariaClassifier.utils.common import read_yaml, create_directories

In [28]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        config = self.config.training 
        params = self.params 
        dataset = os.path.join("artifacts", "data_ingestion", "dataset")
        create_directories([config.root_dir])
        training_config = TrainingConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            epochs=params.EPOCHS,
            imgsz=params.IMG_SIZE,
            batch_size=params.BATCH_SIZE,
            weights=params.WEIGHTS,
            include_top=params.INCLUDE_TOP,
            lr=params.LEARNING_RATE,
            dataset=Path(dataset)

        )
        return training_config


In [7]:
import tensorflow as tf
import numpy as np 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [29]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config 

    def train_val_gen(self):
        train_ds = os.path.join(self.config.dataset, "train")
        val_ds = os.path.join(self.config.dataset, "val")
        train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.2,
                                           shear_range=0.2, horizontal_flip=True)
        val_datagen = ImageDataGenerator(rescale=1./255)
        train_gen = train_datagen.flow_from_directory(train_ds,
                                                      target_size=(self.config.imgsz,self.config.imgsz),
                                                      class_mode='categorical')
        val_gen = val_datagen.flow_from_directory(val_ds,
                                                      target_size=(self.config.imgsz,self.config.imgsz),
                                                      class_mode='categorical')
        
        return train_gen, val_gen
    
    def train(self):
        base_model = tf.keras.applications.Xception(input_shape=(self.config.imgsz,self.config.imgsz,3),
                                               include_top=self.config.include_top,
                                               weights=self.config.weights)
        base_model.trainable = False
        model = tf.keras.Sequential([
            base_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(2, activation='softmax')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.lr),
                      loss = 'categorical_crossentropy',
                      metrics=['accuracy'])
        train_gen, val_gen = self.train_val_gen()
        model.fit(train_gen, epochs=self.config.epochs,
                  validation_data=val_gen,
                  steps_per_epoch=len(train_gen)/self.config.batch_size,
                  validation_steps=len(val_gen)/self.config.batch_size)
        model.save(self.config.model_path)


In [30]:
try:
    config = ConfigurationManager()
    train_config = config.get_training_config()
    training = Training(config=train_config)
    training.train()
except Exception as e:
    raise e

[2024-01-26 14:31:14,101: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-26 14:31:14,193: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-26 14:31:14,195: INFO: common: created directory at: artifacts]
[2024-01-26 14:31:14,240: INFO: common: created directory at: artifacts/training]
Found 19290 images belonging to 2 classes.
Found 5512 images belonging to 2 classes.


2024-01-26 14:31:24.719199: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


38/37 [==============================] - ETA: -3s - loss: 0.3428 - accuracy: 0.8520

2024-01-26 14:39:32.924558: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


37/37 [==============================] - 623s 17s/step - loss: 0.3428 - accuracy: 0.8520 - val_loss: 0.2380 - val_accuracy: 0.9034
